In [0]:
#display the CVS file is uploaded at DBFS storage (internal to Databricks)
display(dbutils.fs.ls('/FileStore/mvp/airbnb'))

path,name,size,modificationTime
dbfs:/FileStore/mvp/airbnb/listings.csv,listings.csv,78951368,1743891667000


In [0]:
 
%sql
--drop database schema airbnb if it is created
-- create the database airbnb, (gold tier) where all study will carried out
DROP SCHEMA IF EXISTS airbnb CASCADE;
CREATE DATABASE airbnb;


In [0]:

from pyspark.sql.types import BooleanType, DoubleType
from pyspark.sql.functions import translate, col, isnan, when, count

#Load CVS into a dataframe. CSV has specifal attributes such as comma as seprator, quotes under fields and line breaks
#Some of the descriptions and names are in PT-BR, so setting encoding to UTF-8
bronze_df = spark.read.csv("dbfs:/FileStore/mvp/airbnb/listings.csv", 
                           header=True, 
                           inferSchema=True, 
                           sep=',', quote='"', 
                           escape='"', 
                           multiLine=True,
                           encoding='UTF-8')

'''
select columns that are part of the assigment. Thus, they will be part of the gold database. This is only for simplicity when 
handling or visualizing data.
'''
SelectedColumns = [
'id',
'name',
'description',
'neighborhood_overview',
'host_name',
'host_since',
'neighbourhood',
'neighbourhood_cleansed',
'latitude',
'longitude',
'property_type',
'room_type',
'price',
'review_scores_accuracy',
'accommodates',
'has_availability']

silver_df = bronze_df[SelectedColumns]

display(silver_df.select([count(when(col(field).isNull(), field)).alias(field) for field in silver_df.columns]))



id,name,description,neighborhood_overview,host_name,host_since,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,review_scores_accuracy,accommodates,has_availability
0,0,1120,22619,2,2,22619,0,0,0,0,0,2855,10420,0,497


In [0]:
#cast the has_availability filed from String to Boolean and make sure there are no NULL values in it
silver_df = silver_df \
        .withColumn('has_availability',silver_df.has_availability.cast(BooleanType())) \
        .dropna(subset='has_availability')

#Null fields in reviews, set to 0
silver_df = silver_df.select(col("*"), when(silver_df.review_scores_accuracy.isNull() , 0) \
        .otherwise(silver_df.review_scores_accuracy).alias("reviews"))
                          
# make the price field to be calculated to extract averages and sums. 
# It contains, on orginal CSV, the symbol "$" and thousand separators which prevents the calculations  
# Cast from String to Double and remove the records which the price field is null, to avoid miscalculations
silver_df = silver_df \
        .withColumn('price', translate('price', "$", "")) \
        .withColumn('price', translate('price', ",", "")) \
        
gold_df = silver_df.withColumn('price', silver_df.price.cast(DoubleType())) \
        .dropna(subset='price')

#create the table listings under airbnb database schema
gold_df.write.mode("overwrite").saveAsTable("airbnb.listings")


In [0]:
%sql 
SELECT * FROM airbnb.listings LIMIT 5;

id,name,description,neighborhood_overview,host_name,host_since,neighbourhood,neighbourhood_cleansed,latitude,longitude,property_type,room_type,price,review_scores_accuracy,accommodates,has_availability,reviews
271975,Unbelievable Ocean View Apartment,null,"If you like hiking, you have to get on this track to the top of ""Dois Irmãos"" hill, one of Rio's most beautiful view. It's not difficult and not a long way up there.http://rioforyou.com/If you like trails, Vidigal is the place to stay. One of the most spectacular views of Rio is on the trail to the top of ""Dois Irmãos"", that mountain that seems inaccessible that you see from the beach of arpoador, ipanema and leblon is not as difficult to access as it seems. Staying in the neighborhood of Vidigal you already have access to the trail much easier and even without a guide, asked the friendly residents of the community you will come to the top with ease.http://trilhadoisirmaos.com.br/Public and beautiful Park in Vidigal:Sitiêhttp://www.parquesitie.org/Restaurants in Vidigal:Barlacubaco RestaurantAv. Presidente João Goulart, 686.Art and Theatre:Nós do Morrowww.nosdomorro.com.br",Maria,2011-11-19,"Rio de Janeiro, Brazil",Vidigal,-22.99508,-43.23604,Entire rental unit,Entire home/apt,236.0,4.33,4,true,4.33
272335,EXCELENTE APARTAMENTO EM COPACABANA,"The apartment has 30m2 with connecting bedroom/living room, full bathroom and small kitchen fully equipped. Air conditioning, wi-fi internet, 1 double bed, 1 double sofa bed, 2 blocks from Copacabana beach.",The beauty of my Copacabana beach. This needs no further comment.,Vitor,2013-07-15,"Rio de Janeiro, Brazil",Copacabana,-22.9623,-43.17389,Entire rental unit,Entire home/apt,723.0,4.91,4,true,4.91
272631,Comfort em Copacabana,"The apartment is located in the Shopping dos Antiqueários in Copacabana and has total 24h security.The rooms are cozy and airy with beautiful views of the green, double box bed, fan, air conditioning, cable TV. Bathroom may be shared . In the kitchen, the microwave oven and the refrigerator can be used. Breakfast is served from 8:30 am to 11am.Available: internet , bed linen and towels every 5 days. We offer transfers for payment.",null,Neyde,2011-11-20,null,Copacabana,-22.9675,-43.18666,Private room in rental unit,Private room,1146.0,5.0,2,true,5.0
273261,Amazing Huge Apartment in Leblon,"spacious apartment, with ocean view, super well furnished, 49 inches flat TV, TV in suite 42 inches, wifi throughout the apartment, two refrigerators, laundry machine, fully equipped kitchen,three bedrooms, three bathrooms. Impeccable!",null,Lucas,2011-04-25,null,Leblon,-22.98632,-43.22936,Entire rental unit,Entire home/apt,6742.0,null,6,true,0.0
273363,Lindo quarto sossegado para temporada,We are happy to share our home with all those who arriveOur daily exercise is empathy for respect for differences!We're so grateful to meet people from all over the world,"Santa Teresa is a unique and very interesting place in Rio. It is situated in the central region and high on a hill offering a privileged view of the city, Guanabara Bay and Pāo de Açúcar. This neighborhood seems to have stopped in time, with its old houses, mansions, narrow cobbled streets and the famous cable car that takes locals and tourists downhill.",Casa Da Carmen E,2011-05-04,"Rio de Janeiro, Brazil",Santa Teresa,-22.91825,-43.17872,Private room in home,Private room,61.0,5.0,1,true,5.0


In [0]:
%sql
SELECT count(*) AS TotalListings,
has_availability
FROM airbnb.listings
GROUP BY has_availability; 

TotalListings,has_availability
36205,true
1,false


The query below indicates the proportion of room types in Rio de janeiro. The most common, (80%) are entire room rentals.

Private rooms are little guestrooms separte from the main house.

In [0]:
%sql
SELECT count(id) as TotalRoomTypes, 
concat(round((TotalRoomTypes/(select count(id) from airbnb.listings) * 100),2), "%") as Percentage, 
room_type as RoomTypes 
FROM airbnb.listings 
GROUP BY room_type 
ORDER BY TotalRoomTypes DESC;

TotalRoomTypes,Percentage,RoomTypes
29267,80.83%,Entire home/apt
6663,18.4%,Private room
256,0.71%,Shared room
20,0.06%,Hotel room


Databricks visualization. Run in Databricks to view.

The query below indicates the proportion of listings in each neighbourhood in Rio de janeiro. 

Copacabana outstands as the place where there more listings in Rio. It almost the same amount of the next 5 neighbourhoods added together.

In [0]:
%sql
SELECT count(id) as TotalInNeighbourhoods, 
concat(round((TotalInNeighbourhoods/(select count(id) from airbnb.listings) * 100),2), "%") as Percentage,
neighbourhood_cleansed as Neighbourhood
FROM airbnb.listings 
GROUP BY neighbourhood_cleansed
ORDER BY TotalInNeighbourhoods DESC
LIMIT 10;

TotalInNeighbourhoods,Percentage,Neighbourhood
11421,31.54%,Copacabana
3342,9.23%,Barra da Tijuca
3332,9.2%,Ipanema
1894,5.23%,Recreio dos Bandeirantes
1704,4.71%,Jacarepaguá
1638,4.52%,Leblon
1539,4.25%,Centro
1521,4.2%,Botafogo
1150,3.18%,Santa Teresa
793,2.19%,Flamengo


Databricks visualization. Run in Databricks to view.

The amount of each room type under he same neighborhood

In [0]:
%sql
SELECT
    neighbourhood_cleansed,
    room_type,
    count(*) AS subgroup_count
FROM
    airbnb.listings
GROUP BY
    neighbourhood_cleansed,
    room_type
ORDER BY
   neighbourhood_cleansed
LIMIT 20;

neighbourhood_cleansed,room_type,subgroup_count
Abolição,Private room,1
Abolição,Entire home/apt,4
Alto da Boa Vista,Private room,14
Alto da Boa Vista,Entire home/apt,26
Anchieta,Entire home/apt,1
Andaraí,Private room,15
Andaraí,Entire home/apt,21
Andaraí,Shared room,2
Anil,Entire home/apt,38
Anil,Private room,18


Evaluate the porperty sizes correlation with prices.
Usually, high property prices corresponds the capability to accommodate more people. 
THere are some outliers, like Anchieta that have only 1 listing.

In [0]:
%sql
SELECT round(avg(price), 2) as AveragePrice,
round(avg(accommodates),2) as Accomodates,
round((AveragePrice/Accomodates), 2) as PricePerPerson,
neighbourhood_cleansed as Neighbourhood,
count(id) as TotalListings
FROM airbnb.listings
GROUP BY neighbourhood_cleansed
ORDER BY PricePerPerson DESC
LIMIT 20;

AveragePrice,Accomodates,PricePerPerson,Neighbourhood,TotalListings
9073.54,8.7,1042.94,Joá,155
3581.89,5.11,700.96,São Conrado,314
6068.0,10.0,606.8,Anchieta,1
1878.61,3.52,533.7,Curicica,67
3000.23,5.64,531.96,Itanhangá,158
2032.96,4.12,493.44,Lagoa,257
1890.02,4.08,463.24,Ipanema,3332
1751.0,4.0,437.75,Leblon,1638
1607.08,3.84,418.51,Cachambi,25
1599.79,4.13,387.36,Gávea,205


RevPar (Revenue per Available Room) is a metric used in hospitality to measure the occupancy rate.

it devides the total **room revenue** by the **total number of rooms available** in the period being measured. This indicator brings how much revenue each listing generates in a certain period of time, whether occupied or not.

In order to avoid any distorcions, it was selected only regions of the city of Rio de Janeiro that have more than 500 listings. For the sake of the study of the assignment, each reagion is being regarded as a hotel which all listing available represent the total number of rooms avaible.

In [0]:
%sql
SELECT round((sum(price)/count(id)), 2) as RevPAR,
count(id) as TotalListings,
neighbourhood_cleansed as Neighbourhood
FROM airbnb.listings
WHERE has_availability = TRUE
GROUP BY neighbourhood_cleansed
HAVING count(id) > 500
ORDER BY RevPAR DESC
LIMIT 50;

RevPAR,TotalListings,Neighbourhood
1890.02,3332,Ipanema
1751.0,1638,Leblon
1389.8,3342,Barra da Tijuca
1356.27,11420,Copacabana
1224.24,718,Leme
908.44,1894,Recreio dos Bandeirantes
779.7,1704,Jacarepaguá
762.9,1150,Santa Teresa
715.44,793,Flamengo
703.65,1521,Botafogo


Group each review range and evaluate whether there is an impact on price (revenue). 

Zero replaces all listings with NULL in the CSV file.

The query shows there is a weak relationship between prices and review ratings. The correlation is higher on the occommodation sizes and neighbourhood. 

In [0]:
%sql
select t.range as ReviewRange, count(*) as TotalProperties, round(avg(t.price), 2) as AvgPrice
from (
  select case  
    when reviews between 4.0 and 4.9 then '4-4.9'
    when reviews between 3.0 and 3.9 then '3-3.9'
    when reviews between 2.0 and 2.9 then '2-2.9'
    when reviews between 1.0 and 1.9 then '1-1.9'
    when reviews between 0.1 and 0.9 then '0-0.9'
    when reviews == 5 then '5'
    else 'not defined' end as range,
    price
  from airbnb.listings) t
group by t.range
order by AvgPrice desc



ReviewRange,TotalProperties,AvgPrice
not defined,13954,1516.35
1-1.9,53,1480.0
5,10451,1055.57
4-4.9,11365,1047.5
3-3.9,326,999.05
2-2.9,57,734.93
